Los RDDs casi no se usan para el procesamiento hoy en día, pero son la base de las nuevas interfaces como Spark SQL y Dataframes. Estas herramientas internamente usan RDD.

Cuando escribimos código, este no se ejecuta inmediatamente, se produce un lazy evaluation que se traduce a un plan de ejecución. Este plan solo será ejecutado cuando se realice una ACCIÓN, no TRANSFORMACIONES.

# Introduccion

## Instalación manual

In [ ]:
# Instalar SDK Java 8
!apt-get install openjdk-8-jdk-headless -qq > /dev/null

In [ ]:
# Descargar Spark 3.4.3

!wget -q https://archive.apache.org/dist/spark/spark-3.4.3/spark-3.4.3-bin-hadoop3.tgz

In [ ]:
# Descomprimir el archivo descargado de Spark

!tar xf spark-3.4.3-bin-hadoop3.tgz


In [ ]:
# Establecer las variables de entorno

import os

os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["SPARK_HOME"] = "/content/spark-3.4.3-bin-hadoop3"

In [2]:
# Instalar la librería findspark
!pip install -q findspark

In [ ]:
### verificar la instalación ###

import findspark

findspark.init()

## Instalación por default con pip

In [3]:
# Instalar pyspark
!pip install -q pyspark

In [4]:

from pyspark.sql import SparkSession

spark = SparkSession.builder.master("local[*]").getOrCreate()

In [ ]:
# Probando la sesión de Spark
df = spark.createDataFrame([{"Hola": "Mundo"} for x in range(10)])
df.show(10, False)

+-----+
|Hola |
+-----+
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
|Mundo|
+-----+



In [5]:
spark

# RDD

## Creacion RDD

In [ ]:
spark

In [ ]:
sc = spark.sparkContext

In [ ]:
# Crear un RDD vacío
rdd_vacio = sc.emptyRDD

In [ ]:
## Crear un RDD con parallelize
rdd_vacio3 = sc.parallelize([],3)
rdd_vacio3.getNumPartitions()

3

In [ ]:
rdd = sc.parallelize([1,2,3,4,5])
rdd

ParallelCollectionRDD[8] at readRDDFromFile at PythonRDD.scala:287

In [ ]:
rdd.collect()

[1, 2, 3, 4, 5]

In [ ]:
# Crear un RDD desde un archivo de texto

rdd_texto = sc.textFile('/rdd_source.txt')

rdd_texto.collect()

['Así podemos crear', 'un RDD desde un', 'archivo de texto!!!']

In [ ]:
rdd_texto_completo = sc.wholeTextFiles('/rdd_source.txt')

rdd_texto_completo.collect()

[('file:/rdd_source.txt',
  'Así podemos crear\nun RDD desde un\narchivo de texto!!!')]

In [ ]:
rdd_suma = rdd.map(lambda x: x +1)

rdd_suma.collect()

[2, 3, 4, 5, 6]

In [ ]:
df = spark.createDataFrame([(1, 'jose'), (2, 'juan')], ['id', 'nombre'])

df.show()

+---+------+
| id|nombre|
+---+------+
|  1|  jose|
|  2|  juan|
+---+------+



In [ ]:
rdd_df = df.rdd

rdd_df.collect()

[Row(id=1, nombre='jose'), Row(id=2, nombre='juan')]

##Ejercicios


In [ ]:
mysession = SparkSession.builder.master("local[*]").appName("Cap2").getOrCreate()

In [ ]:
mysession

In [ ]:
rdd_vacio = sc.emptyRDD
rdd_5part = sc.parallelize([],5)
rdd_5part.getNumPartitions()

5

In [ ]:
def getNumerosPrimos(a:int):
  result = []
  for i in range(1,a+1):
    flag = True
    for j in range(1,i):
      if(i%j==0 and j!=1):
        flag = False
    if(flag):
      result.append(i)
  return result

In [ ]:
rdd_primos = sc.parallelize(getNumerosPrimos(20))
rdd_primos

ParallelCollectionRDD[39] at readRDDFromFile at PythonRDD.scala:287

In [ ]:
filter_rdd = rdd_primos.filter( lambda a : a>10)
filter_rdd.collect()

[11, 13, 17, 19]

## Transformaciones en RDD

In [6]:
sc = spark.sparkContext

Transformación Map

In [7]:
rdd = sc.parallelize([1,2,3,4,5])

In [11]:
rdd_resta = rdd.map( lambda a: a-1)
rdd_resta.collect()

[0, 1, 2, 3, 4]

In [12]:
rdd_par = rdd.map(lambda a: a%2==0)
rdd_par.collect()

[False, True, False, True, False]

In [13]:
rdd_texto = sc.parallelize(['jose','juan','lucia'])

In [14]:
rdd_mayuscula = rdd_texto.map(lambda a : a.upper())
rdd_mayuscula.collect()

['JOSE', 'JUAN', 'LUCIA']

In [15]:
rdd_hola = rdd_mayuscula.map( lambda a: 'Hola '+ a)
rdd_hola.collect()

['Hola JOSE', 'Hola JUAN', 'Hola LUCIA']

Transformación flatMap

In [16]:
rdd = sc.parallelize([1,2,3,4,5])

In [18]:
rdd_cuadrado = rdd.map(lambda a: (a,a*a))
rdd_cuadrado.collect()

[(1, 1), (2, 4), (3, 9), (4, 16), (5, 25)]

In [19]:
rdd_cuadrado_flat = rdd.flatMap(lambda x  : (x,x**2))
rdd_cuadrado_flat.collect()

[1, 1, 2, 4, 3, 9, 4, 16, 5, 25]

In [21]:
rdd_texto = sc.parallelize(['jose','juan','lucia'])
rdd_mayuscula = rdd_texto.flatMap( lambda x : (x, x.upper()))
rdd_mayuscula.collect()

['jose', 'JOSE', 'juan', 'JUAN', 'lucia', 'LUCIA']

Filter

In [22]:
rdd = sc.parallelize([1,2,3,4,5,6,7,8,9])

In [23]:
rdd_par = rdd.filter(lambda a: a%2==0)
rdd_par.collect()

[2, 4, 6, 8]

In [24]:
rdd_texto = sc.parallelize(['jose','juaquin','juan','lucia','karla'])

In [25]:
rdd_k = rdd_texto.filter(lambda x: x.startswith('k'))
rdd_k.collect()

['karla']

In [27]:
rdd_filtro = rdd_texto.filter(lambda x: x.startswith('j') and x.find('u')==1)
rdd_filtro.collect()

['juaquin', 'juan']

coalesce  (operación costosa)

In [28]:
rdd = sc.parallelize([1,2,3.4,4,5],10)
rdd.getNumPartitions()


10

In [30]:
rdd5 = rdd.coalesce(5)
rdd5.getNumPartitions()

5

Repartition (Operación costosa)

In [31]:
rdd = sc.parallelize([1,2,3,4,5],3)
rdd.getNumPartitions()

3

In [32]:
rdd7= rdd.repartition(7)
rdd7.getNumPartitions()

7

ReducebyKey

In [33]:
rdd = sc.parallelize(
    [('casa', 2),
     ('parque', 1),
     ('que', 5),
     ('casa', 1),
     ('escuela', 2),
     ('casa', 1),
     ('que', 1)]
)

In [34]:
rdd.collect()

[('casa', 2),
 ('parque', 1),
 ('que', 5),
 ('casa', 1),
 ('escuela', 2),
 ('casa', 1),
 ('que', 1)]

In [35]:
rdd_reducido = rdd.reduceByKey(lambda a,b: a+b)
rdd_reducido.collect()

[('casa', 4), ('parque', 1), ('que', 6), ('escuela', 2)]

## Acciones en un RDD

Reduce

In [36]:
rdd = sc.parallelize([2,4,6,8])
rdd.reduce(lambda x,y: x + y)

20

In [37]:
rdd1 = sc.parallelize([1,2,3,4])
rdd1.reduce(lambda x,y: x * y)

24

Count

In [42]:
rdd = sc.parallelize(['a','b','e','l'])
rdd.count()

4

In [39]:
rdd1 = sc.parallelize([item for item in range(10)])
rdd1.count()

10

Collect

In [43]:
rdd1 = sc.parallelize([(item, item ** 2) for item in range(20)])
rdd1.collect()

[(0, 0),
 (1, 1),
 (2, 4),
 (3, 9),
 (4, 16),
 (5, 25),
 (6, 36),
 (7, 49),
 (8, 64),
 (9, 81),
 (10, 100),
 (11, 121),
 (12, 144),
 (13, 169),
 (14, 196),
 (15, 225),
 (16, 256),
 (17, 289),
 (18, 324),
 (19, 361)]

take, max, saveasTextFile

In [46]:
# take
rdd = sc.parallelize('La programación es bella'.split(' '))
rdd.take(2)

['La', 'programación']

In [47]:
# max
rdd1 = sc.parallelize([item/(item + 1) for item in range(10)])
rdd1.max()

0.9

In [48]:
# saveAsTextFile
rdd.collect()

['La', 'programación', 'es', 'bella']

In [49]:
rdd.saveAsTextFile('./rdd')
rdd.coalesce(1).saveAsTextFile('./rdd1')

# Algunos conceptos con RDD

Memoria en cache

In [5]:
rdd = sc.parallelize([item for item in range(10)])
from pyspark.storagelevel import StorageLevel

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [8]:
rdd.persist(StorageLevel.MEMORY_ONLY)
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [9]:
rdd.persist(StorageLevel.DISK_ONLY)
rdd.unpersist()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

In [10]:
rdd.cache()

ParallelCollectionRDD[0] at readRDDFromFile at PythonRDD.scala:289

Shuffling  
(Movimiento de datos necesario para el reparticionamiento)

BroadCast Variables

In [17]:
rdd = sc.parallelize([item for item in range(10)])
uno = 1
br_uno = sc.broadcast(uno)

In [12]:
rdd1 = rdd.map(lambda x: x + br_uno.value)
rdd1.collect()


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [14]:
## BroadCast ocupan memoria en todos los ejecutores
## Eliminar datos de broadcast de cache, eliminar de los ejecutores
br_uno.unpersist()
rdd1  = rdd.map(lambda x: x + br_uno.value)
rdd1.collect()


[1, 2, 3, 4, 5, 6, 7, 8, 9, 10]

In [18]:
br_uno.destroy()

Acumuladores

In [19]:
rdd1 = sc.parallelize('Mi nombre es Jose Miguel y me siento genial'.split(' '))
acumulador1 = sc.accumulator(0)
rdd1.foreach(lambda x: acumulador1.add(1))
print(acumulador1.value)

9
